In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
#Install the forked cell-load
#!pip install --upgrade git+https://github.com/rkita/cell-load.git@scplode-integration
#%pip install -q git+https://github.com/rkita/cell-load.git@scplode-integration


In [45]:
#Download Competition Support Set

In [46]:
from cell_load.data_modules import PerturbationDataModule
import toml

In [47]:
#Make sure the version of PerturbationDataModule  is the right one
import inspect
sig = inspect.signature(PerturbationDataModule.__init__)
assert 'use_scplode' in sig.parameters

In [48]:
config = {
    "datasets": {
        "h1": "/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5"
    },
    "training": {
        "h1": "train"
    },
    "zeroshot": {},
    "fewshot": {}
}

with open("config.toml", "w") as f:
    toml.dump(config, f)

In [49]:
#Prepare the data upfront
import scplode as sp
sdata = sp.read_h5ad("/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5")
sdata[0:10]

[INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)


View of AnnData object with n_obs × n_vars = 10 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'batch_var', 'cell_type'

In [50]:
%%time

import tracemalloc

tracemalloc.start()

dm = PerturbationDataModule(
    toml_config_path="config.toml",
    embed_key=None,
    num_workers=4,
    batch_col="batch",
    cell_sentence_len=16,
    pert_col="target_gene",
    cell_type_key="cell_type",
    control_pert="non-targeting",
    use_scplode = True,
    output_space = "all"
)
dm.setup()

# # Get training data
train_loader = dm.train_dataloader()

for i, batch in enumerate(train_loader):
    break  # Just test the first batch

#Assess peak memory
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak = peak / 1024**2  # MB
print(f"peak memory: {peak:.2f} MB")

/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5


Processing h1:   0%|                                                                                         | 0/1 [00:00<?, ?it/s][INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)
Processing h1: 100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.81it/s]


Processed competition_train: 221273 train, 0 val, 0 test
peak memory: 139.50 MB
CPU times: user 13.4 s, sys: 586 ms, total: 13.9 s
Wall time: 34.8 s


In [51]:
%%time

import tracemalloc

tracemalloc.start()

dm = PerturbationDataModule(
    toml_config_path="config.toml",
    embed_key=None,
    num_workers=4,
    batch_col="batch",
    cell_sentence_len=16,
    pert_col="target_gene",
    cell_type_key="cell_type",
    control_pert="non-targeting",
    use_scplode = False,
    output_space = "all"
)
dm.setup()

# # Get training data
train_loader = dm.train_dataloader()

for i, batch in enumerate(train_loader):
    break  # Just test the first batch

#Assess peak memory
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak = peak / 1024**2  # MB
print(f"peak memory: {peak:.2f} MB")

/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5


Processing h1: 100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.74it/s]


Processed competition_train: 221273 train, 0 val, 0 test
peak memory: 114.58 MB
CPU times: user 13 s, sys: 502 ms, total: 13.5 s
Wall time: 57.3 s
